### Loading Data

In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
data  = pd.read_csv("C:\\Users\\DEBKRISHNA\\Downloads\\archive\\deliveries.csv")
#data["batsman"].unique()

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\DEBKRISHNA\AppData\Local\Continuum\anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
#data["bowler"].unique()

In [3]:
data1 = data[((data["batsman"] ==  "SC Ganguly") | (data["batsman"] == "SK Raina")) & ((data["bowler"] == "P Kumar") | (data["bowler"] == "B Lee")) ]

In [4]:
data2 = data1[["batsman","bowler","over","total_runs"]]


In [5]:
data3 = data[(data["batsman"] ==  "SC Ganguly")  &  (data["bowler"] == "P Kumar") ]
data3 

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,1,SC Ganguly,BB McCullum,P Kumar,0,...,0,1,0,0,0,1,1,NaN,NaN,NaN
13,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,3,1,SC Ganguly,BB McCullum,P Kumar,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
14,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,3,2,SC Ganguly,BB McCullum,P Kumar,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
15,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,3,3,SC Ganguly,BB McCullum,P Kumar,0,...,0,1,0,0,0,1,1,NaN,NaN,NaN
18,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,3,6,SC Ganguly,BB McCullum,P Kumar,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
26,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,5,1,SC Ganguly,BB McCullum,P Kumar,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
27,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,5,2,SC Ganguly,BB McCullum,P Kumar,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
31,1,1,Kolkata Knight Riders,Royal Challengers Bangalore,5,6,SC Ganguly,BB McCullum,P Kumar,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
6641,29,1,Kolkata Knight Riders,Royal Challengers Bangalore,5,1,SC Ganguly,DJ Hussey,P Kumar,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
6642,29,1,Kolkata Knight Riders,Royal Challengers Bangalore,5,2,SC Ganguly,DJ Hussey,P Kumar,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


### Train -Test split

In [6]:
np.random.seed(2)
data3_shuffle = data3.iloc[np.random.permutation(len(data3))]
training_set = data3_shuffle[:31]
test_set = data3_shuffle[31:]
training_set["total_runs"].value_counts()


0    15
1    10
4     5
6     1
Name: total_runs, dtype: int64

### Parameter Estimation

In [7]:
cricket_obs = np.array([15,10,0,0,5,0,1])
def parameter_est(data):
    a,b =0,1
    x,y = 1,1
    with pm.Model() as cricket_model:
        sigma2  = pm.Gamma("sigma2",x,y)
        d1  = pm.Uniform("d1",a,b)
        d2  = pm.Uniform("d2",a,b)
        a_l0 = pm.Normal("a_l0",0,sigma2)
        a_l1 = pm.Normal("a_l1",0,sigma2)
        a_l2 = pm.Normal("a_l2",0,sigma2)
        a_l3 = pm.Normal("a_l3",0,sigma2)
        a_l4 = pm.Normal("a_l4",0,sigma2)
        a_l5 = pm.Normal("a_l5",0,sigma2)   
        a_l6 = pm.Normal("a_l6",0,sigma2)     
        mu1 = pm.Normal("mu1",0,sigma2)
        mu2 = pm.Normal("mu2",0,sigma2)

        p0 = pm.Deterministic('p0', 1. / (1. + np.exp(a_l0-mu1+mu2+d1+d2)))
        p1 = pm.Deterministic('p1', 1. / (1. + np.exp(a_l1-mu1+mu2+d1+d2)))
        p2 = pm.Deterministic('p2', 1. / (1. + np.exp(a_l2-mu1+mu2+d1+d2)))
        p3 = pm.Deterministic('p3', 1. / (1. + np.exp(a_l3-mu1+mu2+d1+d2)))
        p4 = pm.Deterministic('p4', 1. / (1. + np.exp(a_l4-mu1+mu2+d1+d2)))
        p5 = pm.Deterministic('p5', 1. / (1. + np.exp(a_l5-mu1+mu2+d1+d2)))
        p6 = pm.Deterministic('p6', 1. / (1. + np.exp(a_l6-mu1+mu2+d1+d2)))

        observed = pm.Multinomial('obs',n = sum(data),p = [p0,p1,p2,p3,p4,p5,p6] , observed=data)
        step = pm.Metropolis()
        cricket_trace = pm.sample(1000, step=step);
        mu1_est  = cricket_trace["mu1"].mean()
        mu2_est  = cricket_trace["mu2"].mean()
        d1_est   = cricket_trace["d1"].mean()
        d2_est   = cricket_trace["d2"].mean()
        a_l0_est = cricket_trace["a_l0"].mean()
        a_l1_est = cricket_trace["a_l1"].mean()
        a_l2_est = cricket_trace["a_l2"].mean()
        a_l3_est = cricket_trace["a_l3"].mean()
        a_l4_est = cricket_trace["a_l4"].mean()
        a_l5_est = cricket_trace["a_l5"].mean()
        a_l6_est = cricket_trace["a_l6"].mean()
        
        return (mu1_est,mu2_est,d1_est,d2_est,a_l0_est,a_l1_est,a_l2_est,a_l3_est,a_l4_est,a_l5_est,a_l6_est)
    

In [8]:
parameter_est(cricket_obs)

Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [mu2]
>Metropolis: [mu1]
>Metropolis: [a_l6]
>Metropolis: [a_l5]
>Metropolis: [a_l4]
>Metropolis: [a_l3]
>Metropolis: [a_l2]
>Metropolis: [a_l1]
>Metropolis: [a_l0]
>Metropolis: [d2]
>Metropolis: [d1]
>Metropolis: [sigma2]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 123 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


(-1.1462012576894138,
 0.9409165675530644,
 0.5518892510376349,
 0.5327360170291006,
 -2.603074584615584,
 -1.8992952593071553,
 2.2896767763748467,
 2.3171114420637724,
 -0.8097484292514637,
 2.369657543470292,
 1.0925499013090854)

### Checking Model Adequacy on test-set

In [9]:
test_set["total_runs"].value_counts()

0    5
1    4
4    1
Name: total_runs, dtype: int64

#### Total no. estimated (by Model) frequency of zero run

In [10]:
10*(1. / (1. + np.exp( -2.603074584615584+1.1462012576894138+0.9409165675530644+0.5518892510376349+ 0.5327360170291006)))

3.615441162517092

we can see original frequency is 5 which is almost same as estimated value fiited by model

#### Total no. estimated (by Model) frequency of one run

In [11]:
10*(1. / (1. + np.exp( -1.8992952593071553+1.1462012576894138+0.9409165675530644+0.5518892510376349+ 0.5327360170291006)))

2.1883851124540907

we can see original frequency is 4 which is almost same as estimated value fiited by model

#### Total no. estimated (by Model) frequency of two runs

In [12]:
10*(1. / (1. + np.exp( 2.2896767763748467+1.1462012576894138+0.9409165675530644+0.5518892510376349+ 0.5327360170291006)))

0.042295554121135046

we can see original frequency is 0 which is almost same as estimated value fiited by model

#### Total no. estimated (by Model) frequency of three runs

In [13]:
10*(1. / (1. + np.exp( 2.3171114420637724+1.1462012576894138+0.9409165675530644+0.5518892510376349+ 0.5327360170291006)))

0.041155672915746455

we can see original frequency is 0 which is almost same as estimated value fiited by model

#### Total no. estimated (by Model) frequency of four runs

In [14]:
10*(1. / (1. + np.exp( -0.8097484292514637+1.1462012576894138+0.9409165675530644+0.5518892510376349+ 0.5327360170291006)))

0.86117082925107

we can see original frequency is 1 which is almost same as estimated value fiited by model

#### Total no. estimated (by Model) frequency of six runs

In [15]:
10*(1. / (1. + np.exp(  1.0925499013090854+1.1462012576894138+0.9409165675530644+0.5518892510376349+ 0.5327360170291006)))

0.1386681296618805

we can see original frequency is 0 which is almost same as estimated value fiited by model